In [1]:
import pandas as pd
import numpy as np
import cobra
import re

from troppo.omics.readers.generic import TabularReader
from troppo.methods_wrappers import ModelBasedWrapper, ReconstructionWrapper
from troppo.omics.integration import ContinuousScoreIntegrationStrategy
from troppo.methods.reconstruction.gimme import GIMME, GIMMEProperties
patt = re.compile('__COBAMPGPRDOT__[0-9]{1}')
replace_alt_transcripts = lambda x: patt.sub('', x)
# def replace_alt_transcripts(x):
#     # remove COBAMP transcript suffixes like __COBAMPGPRDOT__12
#     x = re.sub(r'__COBAMPGPRDOT__\d+', '', x)
#     # remove leading underscores before ENSG IDs
#     x = re.sub(r'_(?=ENSG)', '', x)
#     # insert missing spaces around 'and'/'or' if they’re glued to gene IDs
#     x = re.sub(r'(?i)([A-Z0-9])(?=and)', r'\1 ', x)
#     x = re.sub(r'(?i)(and)(?=[A-Z0-9])', r'\1 ', x)
#     x = re.sub(r'(?i)([A-Z0-9])(?=or)', r'\1 ', x)
#     x = re.sub(r'(?i)(or)(?=[A-Z0-9])', r'\1 ', x)
#     # ensure lowercase boolean operators and single spaces
#     x = re.sub(r'\bAND\b', 'and', x)
#     x = re.sub(r'\bOR\b', 'or', x)
#     x = re.sub(r'\s+', ' ', x).strip()
#     return x
# load model and expression data
model = cobra.io.read_sbml_model('/home/biodata/aman/Human-GEM/model/Human-GEM.xml')
expression_data = pd.read_csv('/home/biodata/aman/expression_data_ensembl.csv', index_col=0)
print(expression_data.head())

expression_data_transposed = expression_data.T

print("\nTransposed shape:", expression_data_transposed.shape)
print(expression_data_transposed.head())

omics_container = TabularReader(path_or_df=expression_data_transposed, 
                                nomenclature='gene',
                                omics_type='transcriptomics').to_containers()

single_sample = omics_container[0]
model_wrapper = ReconstructionWrapper(model=model, ttg_ratio=9999,
                                      gpr_gene_parse_function=replace_alt_transcripts)

data_map = single_sample.get_integrated_data_map(model_reader=model_wrapper.model_reader,
                                                 and_func=min, or_func=sum)
print(list(data_map.get_scores().items())[:10])
def score_apply(reaction_map_scores):
    return {k:0  if v is None else v for k, v in reaction_map_scores.items()}

continuous_integration = ContinuousScoreIntegrationStrategy(score_apply=score_apply)
scores = continuous_integration.integrate(data_map=data_map)
model = model_wrapper.model_reader.model

# search for biomass in reaction names
for r in model.reactions:
    if 'med' in r.name.lower() or 'growth' in r.name.lower():
        print(r.id, r.name)
# print(model_wrapper.model_reader.r_ids)
# Get the index of the biomass reaction in the model. This will be used as objective for the GIMME algorithm.
idx_objective = model_wrapper.model_reader.r_ids.index('MAR13082')
# Create the properties for the GIMME algorithm.
properties = GIMMEProperties(exp_vector=[v for k, v in scores.items()], obj_frac=0.1, objectives=[{idx_objective: 1}],
                             preprocess=True, flux_threshold=0.25, solver='CPLEX',
                             reaction_ids= model_wrapper.model_reader.r_ids, metabolite_ids=model_wrapper.model_reader.m_ids)

# Run the GIMME algorithm.
gimme = GIMME(S=model_wrapper.S, lb=model_wrapper.lb, ub=model_wrapper.ub, properties=properties)

gimme_run = gimme.run()
print(type(gimme_run))
print(gimme_run)
pd.DataFrame(gimme_run).to_csv('gimme_output_tinit.tsv', sep='\t')
#---
# model.reactions
# len(model.reactions)
selected_reactions = [model.reactions[i] for i in gimme_run]
print(selected_reactions)
#---
ctx_model = model.copy()

selected_ids = [r.id for r in selected_reactions]
to_remove = [r for r in ctx_model.reactions if r.id not in selected_ids]

ctx_model.remove_reactions(to_remove, remove_orphans=True)

print(ctx_model)
print(len(model.reactions), len(ctx_model.reactions))
# export
cobra.io.write_sbml_model(ctx_model, "gimme_context_specific_model.xml")
solution = ctx_model.optimize()
print(solution.objective_value)
print(ctx_model.summary())
fluxes = gimme.sol.__dict__['_Solution__value_map']
print(fluxes)

/home/biodata/miniconda3/envs/cplex_aman/lib/python3.10/site-packages/cobamp/wrappers/external_wrappers.py:9: UserWarning: 
The wrappers.external_wrappers module will be deprecated in a future release in favour of the wrappers module. 
    Available ModelObjectReader classes can still be loaded using cobamp.wrappers.<class>. An appropriate model 
    reader can also be created using the get_model_reader function on cobamp.wrappers
  warnings.warn(
